# Conversational AI Back-End Service

This is a sandbox notebook for developing and testing solutions for each step of the exercise before going to the final production ready code.

In [26]:
import pandas as pd
import json
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

load_dotenv()

True

In [35]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [36]:
patients_df = pd.DataFrame(columns=['full_name', 'phone_number', 'birth_date'])
patients_df.to_csv('data/patients_info.csv', index=False)

appointments_df = pd.DataFrame(columns=['patient_name', 'doctor_name', 'date', 'time', 'status'])
appointments_df.to_csv('data/appointments_info.csv', index=False)

In [37]:
pd.read_csv('data/patients_info.csv')

,full_name,phone_number,birth_date


In [38]:
pd.read_csv('data/appointments_info.csv')

,patient_name,doctor_name,date,time,status


In [31]:
def validate_patient(message: str) -> str:
    """
    Function used to register and validate patients info

    Args:
        message: the message sent by the user
    """

    try:
        patient_json = json.loads(str(message).replace('\'', '\"'))

        patients_df = pd.read_csv('data/patients_info.csv')
        df_columns = patients_df.columns
        register_df = pd.DataFrame([[patient_json[df_columns[0]], patient_json[df_columns[1]], patient_json[df_columns[2]]]], columns=df_columns)
        
        patients_df = pd.concat([patients_df, register_df]).drop_duplicates(keep='first')
        patients_df.to_csv('data/patients_info.csv', index=False)

        return 'Success'
        
    except Exception as e:
        print(message)
        print(e)
        return 'Error'

def new_appointment(message: str) -> str:
    """
    This agent is used to answer the question using data from a patients dataset. 

    Args:
        message: the message sent by the user
    """

    try:
        appointment_json = json.loads(str(message).replace('\'', '\"'))

        appointment_df = pd.read_csv('data/appointments_info.csv')
        df_columns = appointment_df.columns
        register_df = pd.DataFrame([[appointment_json[df_columns[0]], appointment_json[df_columns[1]], appointment_json[df_columns[2]], appointment_json[df_columns[3]], "scheduled"]], columns=df_columns)
        
        appointment_df = pd.concat([appointment_df, register_df]).drop_duplicates(keep='last')
        appointment_df.to_csv('data/appointments_info.csv', index=False)

        return 'Success'
        
    except Exception as e:
        print(message)
        print(e)
        return 'Error'

def list_appointment(message: str) -> str:
    """
    This agent is used to answer the question using data from a patients dataset. 

    Args:
        message: the message sent by the user
    """

    try:
        appointment_json = json.loads(str(message).replace('\'', '\"'))

        appointment_df = pd.read_csv('data/appointments_info.csv')
        appointment_df = appointment_df[appointment_df.patient_name == appointment_json['patient_name']]

        return appointment_df.to_dict('records')
        
    except Exception as e:
        print(message)
        print(e)
        return 'Error'
    
def confirm_appointment(message: str) -> str:
    """
    This agent is used to answer the question using data from a patients dataset. 

    Args:
        message: the message sent by the user
    """

    try:
        appointment_json = json.loads(str(message).replace('\'', '\"'))

        appointment_df = pd.read_csv('data/appointments_info.csv')
        df_columns = appointment_df.columns
        register_df = pd.DataFrame([[appointment_json[df_columns[0]], appointment_json[df_columns[1]], appointment_json[df_columns[2]], appointment_json[df_columns[3]], "confirmed"]], columns=df_columns)
        
        appointment_df = pd.concat([appointment_df, register_df]).drop_duplicates(keep='last', subset=['patient_name', 'doctor_name', 'date', 'time'])
        appointment_df.to_csv('data/appointments_info.csv', index=False)

        return 'Success'
        
    except Exception as e:
        print(message)
        print(e)
        return 'Error'
    
def cancel_appointment(message: str) -> str:
    """
    This agent is used to answer the question using data from a patients dataset. 

    Args:
        message: the message sent by the user
    """

    try:
        appointment_json = json.loads(str(message).replace('\'', '\"'))

        appointment_df = pd.read_csv('data/appointments_info.csv')
        df_columns = appointment_df.columns
        register_df = pd.DataFrame([[appointment_json[df_columns[0]], appointment_json[df_columns[1]], appointment_json[df_columns[2]], appointment_json[df_columns[3]], "canceled"]], columns=df_columns)
        
        appointment_df = pd.concat([appointment_df, register_df]).drop_duplicates(keep=False, subset=['patient_name', 'doctor_name', 'date', 'time'])
        appointment_df.to_csv('data/appointments_info.csv', index=False)

        return 'Success'
        
    except Exception as e:
        print(message)
        print(e)
        return 'Error'

In [32]:
validate_patient({"full_name":"Rafael Barreira","phone_number":"1177665544","birth_date":"1997-01-21"})

pd.read_csv('data/patients_info.csv')

,full_name,phone_number,birth_date
0,Rafael Barreira,1177665544,1997-01-21


In [33]:
new_appointment({"patient_name":"Rafael Barreira","doctor_name":"Dr. Joao Miara","date":"2025-12-01","time":"14:00 pm"})

pd.read_csv('data/appointments_info.csv')

,patient_name,doctor_name,date,time,status
0,Rafael Barreira,Dr. Joao Miara,2025-12-01,14:00 pm,scheduled


In [34]:
list_appointment({"patient_name":"Rafael Barreira"})

[{'patient_name': 'Rafael Barreira',
  'doctor_name': 'Dr. Joao Miara',
  'date': '2025-12-01',
  'time': '14:00 pm',
  'status': 'scheduled'}]

In [10]:
confirm_appointment({"patient_name":"Rafael Barreira","doctor_name":"Dr. Joao Miara","date":"2025-12-01","time":"14:00 pm"})

pd.read_csv('data/appointments_info.csv')

,patient_name,doctor_name,date,time,status
0,Rafael Barreira,Dr. Joao Miara,2025-12-01,14:00 pm,confirmed


In [11]:
cancel_appointment({"patient_name":"Rafael Barreira","doctor_name":"Dr. Joao Miara","date":"2025-12-01","time":"14:00 pm"})

pd.read_csv('data/appointments_info.csv')

,patient_name,doctor_name,date,time,status


In [12]:
class MedicalAgent:
    """
    A class to create and manage a ReAct agent with multiple medical appointment tools.
    """

    _SYSTEM_PROMPT = """
    You are an assistant for medical appointment. 
    When a new conversation begins, the first step might be to validate the users name using the validation tool.
    After that, use the tools to handle the appointments.
    """

    def __init__(self, model: ChatOpenAI):
        """
        Initializes the agent, its tools, and all required components.

        Args:
            model: An instance of a LangChain chat model.
        """

        self.conversation = {'messages': []}

        # Define the list of tools, pointing to the class methods
        tools = [
            self._validation_tool,
            self._new_appointment_tool,
            self._list_appointment_tool,
            self._confirm_appointment_tool,
            self._cancel_appointment_tool
        ]

        # Create and store the final agent
        self.supervisor = create_react_agent(
            model=model, 
            tools=tools, 
            prompt=self._SYSTEM_PROMPT
        )

    def _validation_tool(self, message: str) -> str:
        """
        This tool is used to validate the patients info.
        This tool requires a json format as input
        Required fields:
        - full_name
        - phone_number
        - birth_date
        """

        return validate_patient(message)
    
    def _new_appointment_tool(self, message: str) -> str:
        """
        This tool is used to make a new appointment. This tool requires a json format as input
        Required fields:
        - patient_name
        - doctor_name
        - date
        - time

        """
        print('Tool use')
        return new_appointment(message)
    
    def _confirm_appointment_tool(self, message: str) -> str:
        """
        This tool is used to confirm an appointment. This tool requires a json format as input
        Required fields:
        - patient_name
        - doctor_name
        - date
        - time

        """
        print('Tool use3')
        return confirm_appointment(message)
    
    def _cancel_appointment_tool(self, message: str) -> str:
        """
        This tool is used to cancel an appointment. This tool requires a json format as input
        Required fields:
        - patient_name
        - doctor_name
        - date
        - time

        """
        print('Tool use4')
        return cancel_appointment(message)
    
    def _list_appointment_tool(self, message: str) -> str:
        """
        List all the appointments the patient has.
        Return this to the patient as list and also communicate each corresponding status

        Required fields:
        - patient_name
        """
        print('Tool use 2')
        return list_appointment(message)

    def invoke(self, message: str) -> dict:
        """
        Runs the agent with a user message.

        Args:
            message: The user's input question.

        Returns:
            The final response from the agent.
        """
        self.conversation['messages'].append(('human', message))
        response = self.supervisor.invoke(self.conversation)
        self.conversation = response
        return response

In [ ]:

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

supervisor = MedicalAgent(model=model)

In [14]:
message = "Hello"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

User: Hello
Bot: Hello! To assist you with medical appointments, may I please have your full name, phone number, and birth date for verification?


In [15]:
message = "Rafael Barreira"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

User: Rafael Barreira
Bot: Could you please provide your phone number and birth date as well to complete the verification?


In [16]:
message = "Phone 1177665544 and born in Jan 21 1997"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

User: Phone 1177665544 and born in Jan 21 1997
Bot: Thank you, Rafael Barreira. How can I assist you with your medical appointments today? Would you like to book a new appointment, check existing appointments, confirm, or cancel one?


In [17]:
message = "I need an appointment with Doctor Tony Miaga"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

User: I need an appointment with Doctor Tony Miaga
Bot: Please provide the preferred date and time for your appointment with Doctor Tony Miaga.


In [18]:
message = "Nov 10 2025 at 2pm"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use
User: Nov 10 2025 at 2pm
Bot: Your appointment with Doctor Tony Miaga has been successfully scheduled for November 10, 2025, at 2:00 PM. Is there anything else I can assist you with?


In [19]:
message = "I also need to schedule with Dr. Nancy Nunes, on October 3 2025 at 10 am"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use
User: I also need to schedule with Dr. Nancy Nunes, on October 3 2025 at 10 am
Bot: Your appointment with Dr. Nancy Nunes has been successfully scheduled for October 3, 2025, at 10:00 AM. Would you like to do anything else with your appointments?


In [20]:
message = "Yes, can you list my appointments?"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use 2
User: Yes, can you list my appointments?
Bot: You have the following appointments scheduled:
1. With Doctor Tony Miaga on November 10, 2025, at 2:00 PM. Status: Scheduled.
2. With Doctor Nancy Nunes on October 3, 2025, at 10:00 AM. Status: Scheduled.

Is there anything else you would like to do?


In [21]:
message = "Yes, I want to confirm the appoint with Dr. Tony"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use3
User: Yes, I want to confirm the appoint with Dr. Tony
Bot: Your appointment with Doctor Tony Miaga on November 10, 2025, at 2:00 PM has been confirmed. Is there anything else I can help you with?


In [22]:
message = "Yes, can you list my appointments again?"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use 2
User: Yes, can you list my appointments again?
Bot: Here are your current appointments:
1. With Doctor Nancy Nunes on October 3, 2025, at 10:00 AM. Status: Scheduled.
2. With Doctor Tony Miaga on November 10, 2025, at 2:00 PM. Status: Confirmed.

Let me know if you need any further assistance.


In [23]:
message = "I need to cancel the one from october"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use4
User: I need to cancel the one from october
Bot: Your appointment with Doctor Nancy Nunes on October 3, 2025, at 10:00 AM has been canceled. Is there anything else you would like to do?


In [24]:
message = "List my appointments please"
response = supervisor.invoke(message)

print('User:', message)
print('Bot:', response['messages'][-1].content)

Tool use 2
User: List my appointments please
Bot: You currently have one appointment scheduled:
- With Doctor Tony Miaga on November 10, 2025, at 2:00 PM. Status: Confirmed.

If you need any more assistance, feel free to ask!


In [25]:
response

{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='b47c6dcb-be12-458c-8bb8-855507d70386'),
  AIMessage(content='Hello! To assist you with medical appointments, may I please have your full name, phone number, and birth date for verification?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 338, 'total_tokens': 365, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CNr671XrfNfHxFoVKM0o6x9V09EUK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--03cfac7a-e856-4899-9113-2d0fb2de070a-0', usage_metadata={'input_tokens': 338, 'output_tokens': 27, 'total_tokens': 365, 'input_token_details': {'audio': 0, 'cache